In [1]:
import sys
print(sys.executable)

/grad/sguotong/.conda/envs/.cavGZ_venv/bin/python


In [31]:
from bessel import initialize_bessel_table, BesselJZeros, BesselJpZeros
initialize_bessel_table(10, 10)

# print out a table of BesselJZeros
print("BesselJZeros (m,n): \n")
print("k |  J_0(x) |  J_1(x) |  J_2(x) |  J_3(x) |  J_4(x) |  J_5(x) |\n")
for n in range(1, 6):
    str_out = f"{n} |"
    for m in range(0, 6):
        str_out += f" {BesselJZeros(m, n):.5f} |"

    print(str_out)

print("\n https://mathworld.wolfram.com/BesselFunctionZeros.html")

BesselJZeros (m,n): 

k |  J_0(x) |  J_1(x) |  J_2(x) |  J_3(x) |  J_4(x) |  J_5(x) |

1 | 2.40483 | 3.83171 | 5.13562 | 6.38016 | 7.58834 | 8.77148 |
2 | 5.52008 | 7.01559 | 8.41724 | 9.76102 | 11.06471 | 12.33860 |
3 | 8.65373 | 10.17347 | 11.61984 | 13.01520 | 14.37254 | 15.70017 |
4 | 11.79153 | 13.32369 | 14.79595 | 16.22347 | 17.61597 | 18.98013 |
5 | 14.93092 | 16.47063 | 17.95982 | 19.40942 | 20.82693 | 22.21780 |

 https://mathworld.wolfram.com/BesselFunctionZeros.html


In [ ]:
from cavity_modes import CavityModeHelper
import numpy as np

initialize_bessel_table(m_max=10, n_max=10)

helper = CavityModeHelper(a=0.05, L=0.2, c=1.0)

Er, Ephi, Ez = helper.mode_functions("TM", m=1, n=2, p=0, sign="+")
# Evaluate on arrays; broadcasting works
r = np.linspace(0, 0.05, 200)
phi = np.linspace(0, 2*np.pi, 256)[:, None]   # shape (256, 1)
z = np.linspace(0, 0.2, 128)[None, None, :]   # shape (1, 1, 128)
Er_vals = Er(r[None, :, None], phi, z)        # resulting shape (256, 200, 128)